<a href="https://colab.research.google.com/github/mbakersf/cs1470-linformer/blob/main/Copy_of_CS1470_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup
!pip install fairseq

# Download the IMDB dataset
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar zxvf aclImdb_v1.tar.gz

Streaming output truncated to the last 5000 lines.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/4

In [ ]:
# Format data
import os
import random
from glob import glob

def prepare_data(datadir):
    random.seed(0)
    for split in ['train', 'test']:
        samples = []
        for class_label in ['pos', 'neg']:
            fnames = glob(os.path.join(datadir, split, class_label) + '/*.txt')
            for fname in fnames:
                with open(fname, 'r') as fin:
                    line = fin.readline().strip()
                    samples.append((line, 1 if class_label == 'pos' else 0))
        random.shuffle(samples)
        out_fname = 'train' if split == 'train' else 'dev'
        with open(os.path.join(datadir, out_fname + '.input0'), 'w') as f1, \
             open(os.path.join(datadir, out_fname + '.label'), 'w') as f2:
            for sample in samples:
                f1.write(sample[0] + '\n')
                f2.write(str(sample[1]) + '\n')

prepare_data('aclImdb')

In [ ]:
!git clone https://github.com/pytorch/fairseq
%cd fairseq

Cloning into 'fairseq'...
remote: Enumerating objects: 35184, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 35184 (delta 61), reused 72 (delta 47), pack-reused 35079
Receiving objects: 100% (35184/35184), 25.22 MiB | 23.95 MiB/s, done.
Resolving deltas: 100% (25548/25548), done.
/content/fairseq


In [ ]:
!ls

CODE_OF_CONDUCT.md  fairseq	   LICENSE	   RELEASE.md	     setup.py
CONTRIBUTING.md     fairseq_cli    MANIFEST.in	   release_utils.py  tests
docs		    hubconf.py	   pyproject.toml  scripts	     train.py
examples	    hydra_plugins  README.md	   setup.cfg


In [ ]:

# Download the BPE encoder and vocabulary
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json'
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe'

# BPE encoding of the data
!python -m examples.roberta.multiprocessing_bpe_encoder \
    --encoder-json encoder.json \
    --vocab-bpe vocab.bpe \
    --inputs "../aclImdb/train.input0" "../aclImdb/dev.input0" \
    --outputs "../aclImdb/train.input0.bpe" "../aclImdb/dev.input0.bpe" \
    --workers 60 \
    --keep-empty

# !python -m examples.roberta.multiprocessing_bpe_encoder \
#     --encoder-json encoder.json \
#     --vocab-bpe vocab.bpe \
#     --inputs "aclImdb/train.input0" "aclImdb/dev.input0" \
#     --outputs "aclImdb/train.input0.bpe" "aclImdb/dev.input0.bpe" \
#     --workers 60 \
#     --keep-empty


--2024-05-03 21:23:23--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.11, 3.162.163.34, 3.162.163.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [text/plain]
Saving to: ‘encoder.json’

encoder.json        100%[===================>]   1018K  --.-KB/s    in 0.06s   

2024-05-03 21:23:23 (17.2 MB/s) - ‘encoder.json’ saved [1042301/1042301]

--2024-05-03 21:23:23--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.11, 3.162.163.34, 3.162.163.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================>] 445.62K  -

In [ ]:
# Download the dictionary for fairseq
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt'

# Preprocess the data for fairseq
!fairseq-preprocess \
    --only-source \
    --trainpref "../aclImdb/train.input0.bpe" \
    --validpref "../aclImdb/dev.input0.bpe" \
    --destdir "../IMDB-bin/input0" \
    --srcdict dict.txt \
    --workers 60

!fairseq-preprocess \
    --only-source \
    --trainpref "../aclImdb/train.label" \
    --validpref "../aclImdb/dev.label" \
    --destdir "../IMDB-bin/label" \
    --workers 60


--2024-05-03 21:26:42--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.19, 3.162.163.34, 3.162.163.11, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603290 (589K) [text/plain]
Saving to: ‘dict.txt’

dict.txt            100%[===================>] 589.15K  --.-KB/s    in 0.05s   

2024-05-03 21:26:42 (11.8 MB/s) - ‘dict.txt’ saved [603290/603290]

2024-05-03 21:26:44.846810: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 21:26:44.846867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 21:26:44.848328: E external/loc

In [7]:
# Finetuning the Linformer model
# You will need to have a pre-trained RoBERTa model file 'model.pt'
# !fairseq-train "IMDB-bin/" \
#     --restore-file '/path/to/roberta.large/model.pt' \
#     --max-positions 512 \
#     --batch-size 8 \
#     --max-tokens 4400 \
#     --task sentence_prediction \
#     --reset-optimizer --reset-dataloader --reset-meters \
#     --required-batch-size-multiple 1 \
#     --init-token 0 --separator-token 2 \
#     --arch linformer_roberta_large \
#     --criterion sentence_prediction \
#     --classification-head-name 'imdb_head' \
#     --num-classes 2 \
#     --dropout 0.1 --attention-dropout 0.1 \
#     --weight-decay 0.1 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-06 \
#     --clip-norm 0.0 \
#     --lr-scheduler polynomial_decay --lr 1e-05 --total-num-update 7812 --warmup-updates 469 \
#     --fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
#     --max-epoch 10 \
#     --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
#     --shorten-method "truncate" \
#     --find-unused-parameters \
#     --update-freq 4

!pip install tensorboardX

!fairseq-train "/content/IMDB-bin/" \
    --user-dir /content/fairseq/examples/linformer/linformer_src \
    --max-positions 512 \
    --batch-size 8 \
    --max-tokens 4400 \
    --task sentence_prediction \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --init-token 0 --separator-token 2 \
    --arch linformer_roberta_base \
    --criterion sentence_prediction \
    --classification-head-name 'imdb_head' \
    --num-classes 2 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.1 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-06 \
    --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr 1e-05 --total-num-update 7812 --warmup-updates 469 \
    --fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
    --max-epoch 10 \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --shorten-method "truncate" \
    --find-unused-parameters \
    --update-freq 4

Streaming output truncated to the last 5000 lines.
epoch 007 | valid on 'valid' subset:  81% 2522/3125 [02:40<00:36, 16.36it/s]
epoch 007 | valid on 'valid' subset:  81% 2524/3125 [02:41<00:37, 15.91it/s]
epoch 007 | valid on 'valid' subset:  81% 2526/3125 [02:41<00:35, 16.67it/s]
epoch 007 | valid on 'valid' subset:  81% 2528/3125 [02:41<00:36, 16.15it/s]
epoch 007 | valid on 'valid' subset:  81% 2530/3125 [02:41<00:38, 15.57it/s]
epoch 007 | valid on 'valid' subset:  81% 2532/3125 [02:41<00:38, 15.45it/s]
epoch 007 | valid on 'valid' subset:  81% 2534/3125 [02:41<00:38, 15.41it/s]
epoch 007 | valid on 'valid' subset:  81% 2536/3125 [02:41<00:39, 15.07it/s]
epoch 007 | valid on 'valid' subset:  81% 2538/3125 [02:42<00:39, 14.95it/s]
epoch 007 | valid on 'valid' subset:  81% 2540/3125 [02:42<00:40, 14.62it/s]
epoch 007 | valid on 'valid' subset:  81% 2542/3125 [02:42<00:40, 14.53it/s]
epoch 007 | valid on 'valid' subset:  81% 2544/3125 [02:42<00:39, 14.89it/s]
epoch 007 | valid on 'val

In [ ]:

# Load model and make predictions
from fairseq.models.roberta import RobertaModel

roberta = RobertaModel.from_pretrained(
    'checkpoints',
    checkpoint_file='/content/fairseq/checkpoints/checkpoint_best.pt',
    data_name_or_path='/content/IMDB-bin'
)
roberta.eval()  # disable dropout

# # Example prediction
# tokens = roberta.encode('Best movie this year')
# pred = roberta.predict('imdb_head', tokens).argmax().item()
# print('Prediction:', pred)  # Output: 1 (positive)

ModuleNotFoundError: No module named 'omegaconf'